# Decision Tree and Random Forest from Scratch

## Decision Tree

In [1]:
import numpy as np
from scipy import stats
from abc import ABC, abstractmethod
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    mean_squared_error
)

# TreeNode class represents a single node in the decision tree
class TreeNode:
    def __init__(self):
        self.split_value = None
        self.split_feature = None
        self.left = None
        self.right = None
        self.leaf_value = None

    def set_params(self, split_value, split_feature):
        self.split_value = split_value
        self.split_feature = split_feature

    def set_children(self, left, right):
        self.left = left
        self.right = right

# DecisionTree is an abstract base class for both the classifier and regressor
class DecisionTree(ABC):
    def __init__(self, max_depth=None, min_samples_split=2):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    # Abstract methods for impurity and leaf value calculations
    @abstractmethod
    def _impurity(self, D):
        pass

    @abstractmethod
    def _leaf_value(self, D):
        pass

    # Recursive function to grow the decision tree
    def _grow(self, node, D, level):
        depth = (self.max_depth is None) or (self.max_depth >= (level + 1))
        msamp = (self.min_samples_split <= D.shape[0])
        n_cls = np.unique(D[:, -1]).shape[0] != 1

        if depth and msamp and n_cls:
            min_impurity = float('inf')
            best_feature = None
            best_split = None
            left_D = None
            right_D = None

            for f in range(D.shape[1] - 1):
                for s in np.unique(D[:, f]):
                    D_l = D[D[:, f] <= s]
                    D_r = D[D[:, f] > s]

                    if D_l.size and D_r.size:
                        impurity = (D_l.shape[0] / D.shape[0]) * self._impurity(D_l) + (D_r.shape[0] / D.shape[0]) * self._impurity(D_r)
                        if impurity < min_impurity:
                            min_impurity = impurity
                            best_feature = f
                            best_split = s
                            left_D = D_l
                            right_D = D_r

            node.set_params(best_split, best_feature)
            left_node = TreeNode()
            right_node = TreeNode()
            node.set_children(left_node, right_node)

            self._grow(node.left, left_D, level + 1)
            self._grow(node.right, right_D, level + 1)
        else:
            node.leaf_value = self._leaf_value(D)

    # Recursive function to traverse the decision tree and predict the output
    def _traverse(self, node, X_row):
        if node.leaf_value is None:
            split_value, split_feature = node.split_value, node.split_feature
            if X_row[split_feature] <= split_value:
                return self._traverse(node.left, X_row)
            else:
                return self._traverse(node.right, X_row)
        else:
            return node.leaf_value

    # Train the decision tree using input data X and target labels y
    def train(self, X, y):
        D = np.concatenate((X, y.reshape(-1, 1)), axis=1)
        self.root = TreeNode()
        self._grow(self.root, D, 1)

    # Predict the output for input data X using the trained decision tree
    def predict(self, X):
        return np.array([self._traverse(self.root, X_row) for X_row in X]).flatten()


# DecisionTreeClassifier inherits from the DecisionTree class
class DecisionTreeClassifier(DecisionTree):
    def __init__(self, max_depth=None, min_samples_split=2, criterion='gini'):
        super().__init__(max_depth, min_samples_split)
        self.criterion = criterion

    def _gini(self, D):
        gini = 0
        for c in np.unique(D[:, -1]):
            p = D[D[:, -1] == c].shape[0] / D.shape[0]
            gini += p * (1 - p)
        return gini

    def _entropy(self, D):
        entropy = 0
        for c in np.unique(D[:, -1]):
            p = D[D[:, -1] == c].shape[0] / D.shape[0]
            entropy -= p * np.log2(p)
        return entropy

    def _impurity(self, D):
        if self.criterion == 'gini':
            return self._gini(D)
        elif self.criterion == 'entropy':
            return self._entropy(D)
        else:
            raise ValueError("Invalid criterion specified. Choose either 'gini' or 'entropy'.")

    def _leaf_value(self, D):
        return stats.mode(D[:, -1], keepdims=False)[0]


# DecisionTreeRegressor inherits from the DecisionTree class
class DecisionTreeRegressor(DecisionTree):
    def __init__(self, max_depth=None, min_samples_split=2, criterion='mse'):
        super().__init__(max_depth, min_samples_split)
        self.criterion = criterion

    def _mse(self, D):
        y_mean = np.mean(D[:, -1])
        return np.sum((D[:, -1] - y_mean) ** 2) / D.shape[0]

    def _mae(self, D):
        y_mean = np.mean(D[:, -1])
        return np.sum(np.abs(D[:, -1] - y_mean)) / D.shape[0]

    def _impurity(self, D):
        if self.criterion == 'mse':
            return self._mse(D)
        elif self.criterion == 'mae':
            return self._mae(D)
        else:
            raise ValueError("Invalid criterion specified. Choose either 'mse' or 'mae'.")

    def _leaf_value(self, D):
        return np.mean(D[:, -1])

In [2]:
from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Classification example with the Breast Cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

classifier = DecisionTreeClassifier(max_depth=3, min_samples_split=2, criterion='gini')
classifier.train(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Classification accuracy: {accuracy:.4f}")

# Regression example with the California Housing dataset
housing_data = fetch_california_housing()
X, y = housing_data.data, housing_data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regressor = DecisionTreeRegressor(max_depth=3, min_samples_split=2, criterion='mse')
regressor.train(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.4f}")

Classification accuracy: 0.9240
Mean Squared Error: 0.7361


In this example, we first load the "Breast Cancer" and "California Housing" datasets from scikit-learn. Then we split the data into training and test sets using the train_test_split function.

For classification, we use the DecisionTreeClassifier class with a maximum depth of 3 and the Gini impurity criterion. We train the classifier on the training set and make predictions on the test set. Finally, we calculate the accuracy of the classification.

For the regression, we use the DecisionTreeRegressor class with a maximum depth of 3 and the MSE impurity criterion. We train the regressor on the training set and make predictions on the test set. Finally, we calculate the mean squared error (MSE) to assess the performance of the regression.

We used our decision tree implementation to perform classification with the Breast Cancer dataset and regression with the California Housing dataset. The results are as follows:

Classification accuracy: 0.9240

Mean Squared Error (MSE) for regression: 0.7361.

These results show that our decision tree implementation is able to perform classification and regression tasks on real datasets. However, performance may be lower than scikit-learn's decision tree models due to differences in implementation, special case handling, default settings, and tree stability.

In [3]:
from sklearn.tree import DecisionTreeClassifier as SK_DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor as SK_DecisionTreeRegressor

# Classification example with the Breast Cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

classifier = SK_DecisionTreeClassifier(max_depth=3, min_samples_split=2, criterion='gini')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Scikit-learn DecisionTreeClassifier accuracy: {accuracy:.4f}")

# Regression example with the California Housing dataset
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regressor = SK_DecisionTreeRegressor(max_depth=3, min_samples_split=2, criterion='squared_error')
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Scikit-learn DecisionTreeRegressor Mean Squared Error (MSE): {mse:.4f}")


Scikit-learn DecisionTreeClassifier accuracy: 0.9591
Scikit-learn DecisionTreeRegressor Mean Squared Error (MSE): 0.6325


The performance differences between our decision tree implementation and that of scikit-learn can be explained by several reasons:

Implementation: Scikit-learn is a highly optimized library and uses efficient algorithms to build decision trees. Our implementation is quite simple and might not be as optimized, which can lead to differences in the quality of the trees built and, therefore, in the performance of the model.

Special case handling: Scikit-learn more robustly handles special cases and implementation details, such as balanced splits and stopping conditions. Our simplified implementation may not handle these cases as effectively, which may affect model performance.

Default Settings: The default settings for decision tree models in scikit-learn may be different from the ones we used in our implementation. Differences in parameters, such as maximum depth, minimum number of samples for splitting, and impurity criterion, can affect model performance.

Stability: Decision trees are sensitive to variations in training data. A small change in the training data can result in a very different decision tree. Scikit-learn can use techniques to make the decision tree more stable, which can improve model performance.

These reasons partly explain why the performance of our decision tree implementation may be lower than that of scikit-learn. It is important to note that scikit-learn is a very popular and widely used machine learning library which is designed to be efficient and accurate. Therefore, scikit-learn models are expected to generally provide better performance than implementations made from scratch.

## Random Forest

In [4]:
import numpy as np
from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from scipy.stats import mode

class RandomForest:
    def __init__(self, n_estimators, max_depth=None, min_samples_split=2, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_features = max_features

    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[idxs], y[idxs]

    def _fit_tree(self, tree, X, y):
        if self.max_features is None:
            max_features = int(np.sqrt(X.shape[1]))
        elif self.max_features == "sqrt":
            max_features = int(np.sqrt(X.shape[1]))
        elif self.max_features == "log2":
            max_features = int(np.log2(X.shape[1]))
        elif isinstance(self.max_features, int):
            max_features = self.max_features
        else:
            raise ValueError("Invalid value for max_features.")

        features = np.random.choice(X.shape[1], size=max_features, replace=False)
        tree.fit(X[:, features], y)
        return tree, features

    def train(self, X, y, tree_class):
        self.trees = []
        self.tree_features = []

        for _ in range(self.n_estimators):
            tree = tree_class(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree, features = self._fit_tree(tree, X_sample, y_sample)
            self.trees.append(tree)
            self.tree_features.append(features)

    def predict(self, X):
        predictions = []

        for tree, features in zip(self.trees, self.tree_features):
            predictions.append(tree.predict(X[:, features]))

        return mode(predictions, axis=0, keepdims=False)[0].flatten()


In [5]:
from sklearn.metrics import r2_score

# RandomForestClassifier example with the Breast Cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForest(n_estimators=100, max_depth=3, min_samples_split=2, max_features="sqrt")
rf_classifier.train(X_train, y_train, DecisionTreeClassifier)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Classifier accuracy: {accuracy:.4f}")

# RandomForestRegressor example with the California Housing dataset
data = fetch_california_housing()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_regressor = RandomForest(n_estimators=100, max_depth=3, min_samples_split=2, max_features="sqrt")
rf_regressor.train(X_train, y_train, DecisionTreeRegressor)
y_pred = rf_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Random Forest Regressor Mean Squared Error (MSE): {mse:.4f}")
print(f"Random Forest Regressor R-squared (R^2): {r2:.4f}")



RandomForest Classifier accuracy: 0.9708
RandomForest Regressor Mean Squared Error (MSE): 0.6877
RandomForest Regressor R-squared (R^2): 0.4760


In this example, we used the RandomForest class to perform both classification and regression. For classification, we used the Breast Cancer dataset, and for regression, we used the California Housing dataset. Results are displayed as accuracy for classification and mean squared error (MSE) for regression.

This implementation of Random Forest is based on the implementation of DecisionTreeClassifier and DecisionTreeRegressor that we implemented previously.